## Solving Page Rank Problem

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.linear_model import LinearRegression
import scipy
import time
import re
from scipy.interpolate import make_interp_spline

In [3]:
A_ = pd.read_csv("./ucd/A.txt", header=None)
A = A_.transpose()
U = pd.read_csv("./ucd/U.txt", header=None)
row2del = [1, 68, 149, 323] # there are a few links that are incorrect links
A = A.drop(row2del, axis = 0)
A = A.drop(row2del, axis = 1)
U = U.drop(row2del)

Function that calculate the page rank matrix

In [4]:
# the function to calculate the page rank matrix
def pagerank(A, p = 0.85): 

    n = A.shape[0] # obtain the number of pages 
    
    row_sum = np.asarray(np.sum(A, axis=1)).reshape(n) # obtain the number of nodes

    r_i = [] # obtain the out-degree for each page using for loop

    for i in row_sum:
        if i != 0:
            r_i.append(p/i) 
        else:
            r_i.append(0)
   
    r_i_matrix = np.diag(r_i) # convert the vector to diagnol matrix

    D = r_i_matrix @ A # calculate the density matrix
    
    z = [] # obtain the z vector based on out-degree
    for i in row_sum:
        if i != 0:
            z.append([(1-p)/n])
        else:
            z.append([1/n])
    p = D + np.outer(z,np.ones(n)) # calculate the pagerank matrix

    return p

In [7]:
s_A = scipy.sparse.csr_matrix(A)
pr = pagerank(s_A)

SVD method

In [9]:
def solve_pr_svd(pr): # solving the page rank problem using SVD method.
    n = pr.shape[0] # the number of pages 
    r = np.linalg.matrix_rank(pr)
    R_Y = np.zeros(n) 
    R_Y[0] = 1
    P_t = pr.transpose()
       
    L_P = np.identity(n) - P_t
    
    replace = np.ones(n)
    
    L_P[0,:] = replace
    
    U,s,V=np.linalg.svd(L_P) # obtain the SVD decomposition of L_P

    # solving Ax=b using the equation above
    c = np.dot(U.T, R_Y) # c = U^t*b
    w = np.linalg.solve(np.diag(s),c) # w = V^t*c
    x = np.dot(V.T,w) # x = V*w
    
    return x

In [10]:
t = time.time()
x_d = solve_pr_svd(pr)
s_x_d = np.sort(x_d)[::-1]
print("The time used is", time.time()-t)
print(s_x_d[:20])

The time used is 0.06407046318054199
[0.02547476 0.02404957 0.02238444 0.01145635 0.01111901 0.01059859
 0.00937884 0.00865389 0.00864015 0.00797338 0.00725695 0.00698429
 0.00698318 0.00695704 0.00695194 0.00695194 0.00695194 0.00695194
 0.00695194 0.00695194]


Power method

In [332]:
def solve_pr_power(pr, max_iter = 1000): # solving the page rank problem using SVD method.
    n = pr.shape[0] # the number of pages 
    x = np.ones(n)
    x_pre = np.zeros(n)
    
    for i in range(max_iter):
        
        # caculate x
        x_new = np.absolute(np.dot(pr, x))
        
        # normalize x
        x = x_new / np.linalg.norm(x_new)
        
        if x_pre[n-1] == x [n-1]: # break the loop if it converges
            c_i = i+1
            print("The algorithm converges after "+ str(c_i) +" iterations.")
            break
        else:
            x_pre = x
            
    x /= np.sum(x)
    return x

In [333]:
t = time.time()
x_power = solve_pr_power(pr.T)
s_x_p = np.sort(x_power)[::-1]
print("The time used is", time.time()-t)
print(s_x_p[:20])

The algorithm converges after 179 iterations.
The time used is 0.014404296875
[0.02547476 0.02404957 0.02238444 0.01145635 0.01111901 0.01059859
 0.00937884 0.00865389 0.00864015 0.00797338 0.00725695 0.00698429
 0.00698318 0.00695704 0.00695194 0.00695194 0.00695194 0.00695194
 0.00695194 0.00695194]


In [335]:
top20 = []
for i in range(20):
    index = np.where(x_power == s_x_p[i])
    for j in range(len(index[0])):
        top20.append(index[0][j])
        
U.iloc[top20[:20]]

,0
23,http://ucdavis.edu/help/privacy-accessibility....
368,http://xmlns.com/foaf/0.1
24,http://www.universityofcalifornia.edu
109,http://browsehappy.com
303,http://creativecommons.org/ns#
447,http://disabilities.ucsd.edu
45,http://b
318,http:\/\/schema.org
419,http://students.ucsd.edu
107,http://drupal.org)
